STEP 1:
First we will have to create a table template for our user database. Therefore, we initialize the table first. We have also assigned the size to each cell based on the maximum length of each cell in our excel sheet. 

In [ ]:
#Initialize the Table
import pymysql
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  
cursor = mydb.cursor() 

q = """CREATE TABLE profiles(
   Name varchar(32) NOT NULL,
   Intro varchar(255) NOT NULL,
    Location varchar(32) NOT NULL,
    Job varchar(255) NOT NULL,
    About varchar(5000) NOT NULL,
    Education varchar(5000) NOT NULL,
    Skills varchar(5000) NOT NULL,
    URL varchar(255) NOT NULL
)"""

cursor.execute(q)
    
cursor.close()
mydb.commit()
mydb.close()

STEP 2:
Now we have to insert the data from our excel/csv file to our database table. 

In [15]:
#Insert data into the table

import xlrd
import pymysql

xl_data = xlrd.open_workbook('C:/Users/Sankalp/Desktop/Problem_Sample Linkedin Data.xlsx')
sheet = xl_data.sheet_by_index(0)
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  
cursor = mydb.cursor() 


query = """INSERT INTO profiles (Name, Intro, Location, Job, About, Education, Skills, URL) 
                           VALUES 
                           (%s, %s, %s, %s, %s, %s, %s, %s) """
#n1=n2=n3=n4=n5=n6=n7=n8=0
for i in range(1,sheet.nrows):
    Name = sheet.cell(i,0).value
    Intro = sheet.cell(i,1).value
    Location = sheet.cell(i,2).value
    Job = sheet.cell(i,3).value
    About = sheet.cell(i,4).value
    Education = sheet.cell(i,5).value
    Skills = sheet.cell(i,6).value
    URL = sheet.cell(i,7).value
#     n1 = max(n1,len(Name))
#     n2 = max(n2,len(Intro))
#     n3 = max(n3,len(Location))
#     n4 = max(n4,len(Job))
#     n5 = max(n5,len(About))
#     n6 = max(n6,len(Education))
#     n7 = max(n7,len(Skills))
#     n8 = max(n8,len(URL))
    
    values = (Name,Intro,Location,Job,About,Education,Skills,URL)
    
    cursor.execute(query,values)
#print(n1,n2,n3,n4,n5,n6,n7,n8)    
cursor.close()
mydb.commit()
mydb.close()

STEP 3:
Now we query the database based on the input string 't'. 

In [10]:
import pymysql
from tabulate import tabulate
mydb = pymysql.connect( host = 'localhost' ,  user ="root" ,  passwd = "sankalp1911" , db = "project_db")  

cursor = mydb.cursor() 

t = str(input("Enter a Text Query:"))
f_at = t.find(' at ')
f_in = t.find(' in ')

if((f_at == -1) & (f_in==-1)):
    q = """SELECT * FROM profiles
WHERE Location LIKE '%"""+t+"""%'
   OR Job LIKE '%"""+t+"""%'
   OR Skills LIKE '%"""+t+"""%'"""
    cursor.execute(q)
    res = cursor.fetchall()
    if(len(res) == 0):
        cursor.close()
        quer = """ALTER TABLE profiles ADD FULLTEXT(Location, Job, Skills)"""
        query = """SELECT * FROM profiles WHERE MATCH(Location, Job, Skills)
                AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""
        cursor = mydb.cursor() 
        cursor.execute(quer)
        cursor.execute(query)
        res = cursor.fetchall()
        
        

    
elif((f_at != -1) & (f_in==-1)):
    before_at = t[0:f_at]
    after_at = t[f_at+4:]
    #print(before_at,after_at)
    q = """SELECT * FROM profiles
WHERE ((Job LIKE '%"""+t+"""%')
   OR ((Job LIKE '%"""+before_at+"""%'
   OR Skills LIKE '%"""+before_at+"""%')
   AND Location LIKE '%"""+after_at+"""%'))"""
    cursor.execute(q)
    res = cursor.fetchall()
    
    
elif((f_at == -1) & (f_in!=-1)):
    before_in = t[0:f_in]
    after_in = t[f_in+4:]
    #print(before_in,after_in)
    q = """SELECT * FROM profiles
WHERE ((Location LIKE '%"""+after_in+"""%')
   AND ((Job LIKE '%"""+before_in+"""%'
   OR Skills LIKE '%"""+before_in+"""%')))"""
    cursor.execute(q)
    res = cursor.fetchall()
    
    
else:
    between_at_in = t[f_at+4:f_in]
    after_in = t[f_in+4:]
    before_at = t[0:f_at]
    before_in = t[0:f_in]
    q = """SELECT * FROM profiles
WHERE (Location LIKE '%"""+after_in+"""%'
   AND ((Job LIKE '%"""+before_in+"""%'
   OR Skills LIKE '%"""+before_at+"""%')))"""
    cursor.execute(q)
    res = cursor.fetchall()



# print(l,j)
# quer = """ALTER TABLE profiles ADD FULLTEXT(Location, Job, Skills)"""
# query = """SELECT * FROM profiles WHERE MATCH(Location, Job, Skills)
# AGAINST("""+"\'"+t+"\'"+""" IN BOOLEAN MODE)"""

# cursor.execute(quer)
#cursor.execute(q)

#res = cursor.fetchall()
# print(res[0])
print(str(len(res))+" relevant records found!\n")
#print(tabulate(res, headers=['Name', 'Intro', 'Location', 'Job', 'About', 'Education', 'Skills', 'URL'], tablefmt='psql'))
k=0
for r in res:
    k=k+1
    print("Record:"+str(k))
    print(r)
    print('\n')
cursor.close()
mydb.commit()
mydb.close()

Enter a Text Query:Investor in San Francisco
3 relevant records found!

Record:1
('Gopal PG', 'Investor & Entrepreneur - We are Hiring!', 'San Francisco Bay Area', 'Current: Angel & Venture Investor at Auctoro Partners', "The name of the game is: predictability of revenues; sustainability of the prediction; profitability; and a good de-risking model. Measurement is key to improvement.Putting the corporation's interest ahead of personal interest will advance personal goals in the long term.No single person is indispensable. It is important that you give challenging engagements to deserving people, whether they are young or new in the organization. Youth and empowerment are the keys to scalability and longevity.Every situation is what you make it to be. Confidence is half the battle, and leadership is making the impossible look possible. Speed, imagination and excellence in execution are the only three context-invariant and time-invariant attributes for success.The trust of employees is 

STEP 4: We will develop frontend GUI for the above code

In [36]:
#For GUI

# from tkinter import *
# import pymysql
# from tabulate import tabulate

# class MyWindow:
#     def __init__(self, win):
#         self.lbl1=Label(win, text='Enter the text:')
#         self.lbl2=Label(win, text='Result')
#         self.t1=Entry(bd=3)
#         self.t2=Entry()
#         self.btn1 = Button(win, text='Search')
#         self.lbl1.place(x=100, y=50)
#         self.t1.place(x=200, y=50)
#         self.lbl2.place(x=100, y=100)
#         self.t2.place(x=200, y=100)
#         self.b1=Button(win, text='Search', command=self.search)
#         self.b1.place(x=100, y=150)
#     def search(self):
#         t=str(self.t1.get())
#         cursor = mydb.cursor() 
#         quer = """ALTER TABLE profiles ADD FULLTEXT(Name, Intro, Location, Job, About, Education, Skills, URL)"""
#         query = """SELECT * FROM profiles WHERE MATCH(Name, Intro, Location, Job, About, Education, Skills, URL)
#         AGAINST('Venture capitalists in San Francisco' IN BOOLEAN MODE)"""

#         cursor.execute(quer)
#         cursor.execute(query)

#         res = cursor.fetchall()
#         self.t2.insert(END, str(res))

# window=Tk()
# mywin=MyWindow(window)
# window.title('Hello Python')
# window.geometry("400x300+10+10")
# window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Softwares\Anaconda\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-36-fb5761e0dbd6>", line 25, in search
    cursor.execute(quer)
  File "D:\Softwares\Anaconda\lib\site-packages\pymysql\cursors.py", line 170, in execute
    result = self._query(query)
  File "D:\Softwares\Anaconda\lib\site-packages\pymysql\cursors.py", line 328, in _query
    conn.query(q)
  File "D:\Softwares\Anaconda\lib\site-packages\pymysql\connections.py", line 516, in query
    self._execute_command(COMMAND.COM_QUERY, sql)
  File "D:\Softwares\Anaconda\lib\site-packages\pymysql\connections.py", line 750, in _execute_command
    raise err.InterfaceError("(0, '')")
pymysql.err.InterfaceError: (0, '')
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Softwares\Anaconda\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-inp